In [ ]:
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
import transformers
import os 
import torch

In [ ]:
# Check if cuda is available 
torch.cuda.is_available()

# Pipeline

In [ ]:
# Define Model ID
model_id = "tiiuae/falcon-40b-instruct"
# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
# Load Model 
model = AutoModelForCausalLM.from_pretrained(model_id, cache_dir='./workspace/', 
    torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto", offload_folder="offload")
# Set PT model to inference mode
model.eval()
# Build HF Transformers pipeline 
pipeline = transformers.pipeline(
    "text-generation", 
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

In [ ]:
# Test out the pipeline
pipeline('who is the Dalai Lama?')

# Pass it to Langchain

In [ ]:
# Setup prompt template
template = PromptTemplate(input_variables=['input'], template='{input}') 
# Pass hugging face pipeline to langchain class
llm = HuggingFacePipeline(pipeline=pipeline) 
# Build stacked LLM chain i.e. prompt-formatting + LLM
chain = LLMChain(llm=llm, prompt=template)

In [ ]:
# Test LLMChain 
response = chain.run('who is kim kardashian?')

# Build the App

In [ ]:
# Install Gradio for the UI component
!pip install gradio

In [4]:
# Import gradio for UI
import gradio as gr

In [8]:
chatGPT="The Dalai Lama is a spiritual leader and the highest-ranking monk in Tibetan Buddhism. He is believed by Tibetan Buddhists to be the reincarnation of the previous Dalai Lama and is considered a symbol of compassion, wisdom, and enlightenment."

In [5]:
# Create generate function - this will be called when a user runs the gradio app 
def generate(prompt): 
    # The prompt will get passed to the LLM Chain!
    return chain.run(prompt)
    #return chatGPT
    # And will return responses 

In [6]:
# Define a string variable to hold the title of the app
title = '🦜🔗 Falcon-40b-Instruct'
# Define another string variable to hold the description of the app
description = 'This application demonstrates the use of the open-source `Falcon-40b-Instruct` LLM.'
# pls subscribe 🙏

In [7]:
# Build gradio interface, define inputs and outputs...just text in this
gr.Interface(fn=generate, inputs=["text"], outputs=["text"], 
             # Pass through title and description
             title=title, description=description, 
             # Set theme and launch parameters
             theme='finlaymacklon/boxy_violet').launch(server_port=8080, share=True)

Running on local URL:  http://127.0.0.1:8080
Running on public URL: https://686f87f120ec6e2852.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
